# Импорты

In [20]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm
import lightgbm as lgb 
import optuna
import featuretools as ft
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.preprocessing import PolynomialFeatures

# Импорт данных

In [21]:
data = pd.read_csv('C:\\Users\\akayo\\Hacks\\Alpha_data\\train_1.csv')

In [22]:
data

,target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_491,feature_492,feature_493,feature_494,feature_495,feature_496,feature_497,feature_498,feature_499,feature_500
0,0,train,0,0.372342,1.500852,2.133451,-0.957384,-0.119022,0.122920,0.150993,...,-1.254497,0.0,0.803635,-1.017931,-0.521130,0.647188,1.453860,-0.833480,0.184028,-0.438225
1,0,train,1,0.382215,0.962629,-0.192558,-1.019986,-1.330412,-0.100149,-1.131326,...,-0.884458,0.0,-1.643735,-1.264842,-1.523788,0.604948,0.491954,-0.003686,0.469720,-1.094604
2,0,train,2,0.472528,-0.695334,0.537968,-0.033000,-0.364850,-0.441849,-0.035953,...,-0.322682,0.0,-0.496765,-0.051931,0.743209,-1.395109,0.147941,-0.007553,-0.981296,0.270930
3,0,train,3,0.512260,-1.231960,0.555326,1.456647,1.435358,0.168613,-0.628737,...,0.976566,0.0,1.183049,-0.178662,1.468147,-0.791254,0.000645,0.674715,0.803094,1.037068
4,0,train,4,1.487986,-0.164392,-1.197163,1.549057,0.952673,1.144201,-0.964000,...,-1.655236,0.0,1.076122,-0.555735,1.248728,-0.485943,0.458768,-0.444558,-0.333710,0.380206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449041,0,train,449041,-1.361170,-0.685736,0.270151,0.021570,-0.449457,0.333211,-2.172534,...,0.391052,0.0,1.374081,-0.520653,0.061887,-0.170242,-0.458099,-0.779307,1.204898,-1.146198
449042,0,train,449042,-0.077461,-0.423303,-1.332491,0.938922,0.611769,0.029830,1.003885,...,-0.504832,0.0,-0.960401,0.529591,0.853093,-0.277794,-0.785159,-0.323638,0.170126,-0.441154
449043,0,train,449043,0.862977,-0.878855,-0.540000,1.279250,0.602967,-0.147103,-0.762969,...,-0.007173,0.0,-0.288012,1.073586,1.752900,-0.841066,0.081714,-0.686817,-2.038086,-0.540116
449044,0,train,449044,-1.424335,0.728630,0.088147,-0.344344,2.013250,1.754754,-1.532420,...,0.177312,0.0,1.601795,-0.556962,0.015098,-0.181468,0.844969,0.321016,3.123940,-0.438593


In [37]:
X = data.drop(columns=['id', 'smpl', 'target']).values
y = data['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Функции для препарейшна и идеи

In [ ]:
#PCA + RFE -> 76% (Следует сначала сгенирировать фичи, а затем отобрать) пробуем и featuretools и tsfresh

In [ ]:
def data_preparation(data):
    X = data.drop(columns=['id', 'smpl', 'target']).values
    y = data['target'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=389)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    model = model_lgb = lgb.LGBMClassifier(random_state=42)
    
    rfe = RFE(estimator=model, n_features_to_select=25)
    rfe.fit(X_train, y_train)
    selected_feature_indices = rfe.get_support(indices=True)

    X_train_selected = X_train[:, selected_feature_indices]
    X_test_selected = X_test[:, selected_feature_indices]

    return X_train_selected, X_test_selected

def data_preparation(data):
    X = data.drop(columns=['id', 'smpl', 'target']).values
    y = data['target'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=389)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    return pca

def pca_vizualization(pca):
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_explained_variance = explained_variance_ratio.cumsum()
    
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='--')
    plt.xlabel('Количество компонент')
    plt.ylabel('Доля объясненной дисперсии')
    plt.title('Доля объясненной дисперсии в зависимости от количества компонент')
    plt.grid(True)
    plt.show()
    
    n_components = (cumulative_explained_variance < 0.95).sum() + 1
    print(f'Количество компонент, сохраняющих 95% информации: {n_components}')

In [ ]:
def data_for_lgb(train_data, test_data):
    train_data = lgb.Dataset(
        X_train_selected,
        y_train,
        free_raw_data=False,
    )
    test_data = lgb.Dataset(
        X_test_selected,
        y_test,
        free_raw_data=False,
    )

# Генерация фичей и идеи

### featuretools

In [ ]:
# es = ft.EntitySet(id='data')
# es = es.add_dataframe(dataframe_name='transactions', dataframe=data, index='id')

# agg_primitives = ["sum", "mean", "median", "std", "max", "min"]
# trans_primitives = ["add_numeric", "multiply_numeric", "divide_numeric", "subtract_numeric"]

# feature_matrix, feature_defs = ft.dfs(
#     entityset=es,
#     target_dataframe_name='transactions',
#     max_depth=2,
#     agg_primitives=agg_primitives,
#     trans_primitives=trans_primitives
# )

# print(feature_matrix.head())

#Способ ничего не дал (

### tsfresh

In [ ]:
features = extract_features(data, column_id="id", impute_function=impute)
print(features.head())

### poly features

In [33]:
# poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
# poly_features = poly.fit_transform(data.drop(columns=['id', 'smpl']).values)
# poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(data.columns))

# print(poly_df.head())

#Сделать после отбора фичей

# Отбор фичей и идеи

In [ ]:
model = model_lgb = lgb.LGBMClassifier(random_state=42)

scores = []
n_features = []

# Цикл по количеству признаков
for n in tqdm(range(20, 401, 20)):
    rfe = RFE(model, n_features_to_select=n, step=10)
    
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    X_test_rfe = rfe.transform(X_test)
    
    model.fit(X_train_rfe, y_train)
    
    y_pred = model.predict(X_test_rfe)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    scores.append(accuracy)
    n_features.append(n)

# Построение графика
plt.figure(figsize=(10, 6))
plt.plot(n_features, scores, marker='o')
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Number of Features')
plt.grid(True)
plt.show()

  0%|          | 0/20 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 4125, number of negative: 355111
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.405319 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 118913
[LightGBM] [Info] Number of data points in the train set: 359236, number of used features: 500
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011483 -> initscore=-4.455364
[LightGBM] [Info] Start training from score -4.455364
[LightGBM] [Info] Number of positive: 4125, number of negative: 355111
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.398201 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 118361
[LightGBM] [Info] Number of data points in the train set: 359236, number of used features: 490
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011483 -> initscore=-4.455364
[LightGBM] [Info] Start training from score -4.455364
[Lig

In [ ]:
X_train_selected = pd.read_csv('rfe_train.csv')
X_test_selected = pd.read_csv('rfe_test.csv')

In [ ]:
X_train_selected.columns = np.arange(0, 26)
X_test_selected.columns = np.arange(0, 26)

X_train_selected = X_train_selected.drop(0, axis=1)
X_test_selected = X_test_selected.drop(0, axis=1)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=389)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

model = model_lgb = lgb.LGBMClassifier(random_state=42)

rfe = RFE(estimator=model, n_features_to_select=25)
rfe.fit(X_train, y_train)
selected_feature_indices = rfe.get_support(indices=True)

X_train_selected = X_train[:, selected_feature_indices]
X_test_selected = X_test[:, selected_feature_indices]

In [ ]:
rfe = RFE(estimator=model, n_features_to_select=25, step=10)
rfe.fit(X_train, y_train)
selected_feature_indices = rfe.get_support(indices=True)

X_train_selected = X_train[:, selected_feature_indices]
X_test_selected = X_test[:, selected_feature_indices]

# Закомментированный код

In [ ]:
# model.fit(X_train_selected, y_train)

# y_pred = model.predict_proba(X_test_selected)[:, 1]

# roc_auc = roc_auc_score(y_val, y_pred)

# n_features_list.append(n_features_to_select)
# roc_auc_list.append(roc_auc)

# print(f"Количество признаков: {n_features_to_select}, ROC-AUC: {roc_auc}")

In [ ]:
#Количество признаков: 25, ROC-AUC: 0.8008241473752848 - оптимум